Model 4:
Target
1. Add LR Scheduler 
2. Try Differnet optimizer (Adam)
3. Add Data Augmentation

Result:
1. Parameters : 8024
2. Best Train Accuracy : 99.30
3. Best Test Accuracy : 99.41

Analysis
1. The train and test accuracy increased after adding LR scheduler
2. Accuracy with SGD was 99.35 and improved upon using Adam optimizer. 
2. Addition of data augmentation, reduced initial accuracy but accuracy increased in later epochs. But the test accuracy hovered around 99.38-39
3. Little bit reduction of parameters was bringing down accuracy to around 99.38 so went ahead with 8024 parameters
4. The model is still underfitting as training accuracy has been increasing steadily over epochs and its less than test accuracy

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
train_transforms = transforms.Compose([
    #transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [3]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4459091.79it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 154429.05it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1232255.56it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3764182.72it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

#train_loader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

#test_loader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


In [5]:
dropout_value = 0.1
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    #INPUT
    self.conv1= nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), padding=0, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(8),
        nn.Dropout(dropout_value)
    )#output = 26

    #CONV 1 BLOCK
    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), padding=0, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(dropout_value)
    )#output = 24

    #TRANSITION BLOCK
    self.conv3 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1,1), padding=0, bias=False),
    )#output = 24

    self.pool1 = nn.MaxPool2d(2,2)
    #output = 12

    #CONV 2 BLOCK
    self.conv4 = nn.Sequential(
        nn.Conv2d(in_channels=10, out_channels=16, kernel_size=(3,3), padding=0, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(dropout_value)
    )#output = 10


    self.conv5 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), padding=0, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(dropout_value)
    )#output = 8

    self.conv6 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), padding=0, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(dropout_value)
    )#output = 6


    self.conv7 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1,1), padding=1, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(dropout_value)
    )#output = 6

    #OUTPUT BLOCK
    self.gap = nn.Sequential(
         nn.AvgPool2d(kernel_size=6)
    )#output = 1

    self.conv8 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
    )


  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.pool1(x)
    x = self.conv4(x)
    x = self.conv5(x)
    x = self.conv6(x)
    x = self.conv7(x)
    x = self.gap(x)
    x = self.conv8(x)
    x = x.view(-1,10)
    return F.log_softmax(x, dim=-1)

In [6]:
!pip install torchsummary
from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

model = Net().to(device)
summary(model, input_size=(1,28,28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             160
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,440
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
          Dropout-14           [-1

In [7]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct=0
  processed=0

  for batch_idx, (data,target) in enumerate(pbar):
    data,target = data.to(device), target.to(device)

    optimizer.zero_grad()

    y_pred=model(data)

    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    loss.backward()
    optimizer.step()

    pred = y_pred.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc=f'loss={loss.item()} Batch_id ={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
  model.eval()
  test_loss = 0;
  correct = 0;

  with torch.no_grad():
    for data, target in test_loader:
      data,target = data.to(device), target.to(device)
      output=model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)
  ))

  test_acc.append(100. * correct / len(test_loader.dataset))

In [8]:
from torch.optim.lr_scheduler import StepLR

model = Net().to(device)
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)
EPOCHS = 15

for epoch in range (EPOCHS):
  print("EPOCH: ", epoch+1)
  train(model, device, train_loader, optimizer, epoch)
  scheduler.step()
  test(model, device, test_loader)

EPOCH:  1


loss=0.05267756059765816 Batch_id =468 Accuracy=92.75: 100%|██████████| 469/469 [00:14<00:00, 33.19it/s]  



Test set: Average loss: 0.0708, Accuracy: 9775/10000 (97.75%)

EPOCH:  2


loss=0.05580747500061989 Batch_id =468 Accuracy=97.69: 100%|██████████| 469/469 [00:04<00:00, 105.43it/s] 



Test set: Average loss: 0.0475, Accuracy: 9849/10000 (98.49%)

EPOCH:  3


...


Test set: Average loss: 0.0208, Accuracy: 9938/10000 (99.38%)

EPOCH:  14


loss=0.006257984321564436 Batch_id =468 Accuracy=99.30: 100%|██████████| 469/469 [00:04<00:00, 109.95it/s] 



Test set: Average loss: 0.0207, Accuracy: 9940/10000 (99.40%)

EPOCH:  15


loss=0.007769844960421324 Batch_id =468 Accuracy=99.22: 100%|██████████| 469/469 [00:04<00:00, 110.20it/s] 



Test set: Average loss: 0.0201, Accuracy: 9940/10000 (99.40%)

